In [1]:
!pip install crewai openai requests amadeus tenacity

## Imports and API Setup

In [3]:
import os
import json
import time
import hashlib
import requests
from datetime import datetime
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from tenacity import retry, stop_after_attempt, wait_exponential
from IPython.display import Markdown, display



## API Setup

In [ ]:
OPENAI_API_KEY = "OPENAI_API_KEY"
AMADEUS_CLIENT_ID = "AMADEUS_CLIENT_ID"
AMADEUS_CLIENT_SECRET = "AMADEUS_CLIENT_SECRET"
OPENWEATHER_API_KEY = "OPENWEATHER_API_KEY"
HOTELBEDS_API_KEY = "HOTELBEDS_API_KEY"
HOTELBEDS_SECRET = "HOTELBEDS_SECRET"

# Set environment variables for API usage
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["AMADEUS_CLIENT_ID"] = AMADEUS_CLIENT_ID
os.environ["AMADEUS_CLIENT_SECRET"] = AMADEUS_CLIENT_SECRET
os.environ["OPENWEATHER_API_KEY"] = OPENWEATHER_API_KEY
os.environ["HOTELBEDS_API_KEY"] = HOTELBEDS_API_KEY
os.environ["HOTELBEDS_SECRET"] = HOTELBEDS_SECRET

## Utility and API Functions


This code connects to Amadeus, HotelBeds, and OpenWeather APIs to fetch flight, hotel, and weather information.  
It uses retry logic to handle temporary failures, validates user inputs (dates and airport codes), and builds API requests dynamically based on configuration.  
`generic_api_call()` handles all API calls centrally, minimizing repetition.  
The design ensures reliable, modular, and fault-tolerant data retrieval for travel planning.


In [7]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def get_amadeus_token():
    """Get access token for Amadeus API"""
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": AMADEUS_CLIENT_ID,
        "client_secret": AMADEUS_CLIENT_SECRET
    }
    response = requests.post(url, data=data)
    response.raise_for_status()
    return response.json().get("access_token")

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def get_airport_code(city_name):
    """Fetch airport code for a city using Amadeus API"""
    try:
        token = get_amadeus_token()
        headers = {"Authorization": f"Bearer {token}"}
        params = {"keyword": city_name, "subType": "CITY"}
        url = "https://test.api.amadeus.com/v1/reference-data/locations"
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json().get("data", [])
        return data[0].get("iataCode") if data else None
    except Exception as e:
        print(f"Error fetching airport code for {city_name}: {e}")
        return None

def validate_inputs(inputs):
    """Validate user inputs and add airport codes"""
    inputs = inputs.copy()
    try:
        check_in = datetime.strptime(inputs["check_in_date"], "%Y-%m-%d")
        check_out = datetime.strptime(inputs["check_out_date"], "%Y-%m-%d")
        if check_in < datetime.now() or check_out <= check_in:
            raise ValueError("Invalid dates: Use YYYY-MM-DD, future dates, and check-out date after check-in date.")
        inputs["duration"] = (check_out - check_in).days
    except ValueError:
        raise ValueError("Invalid date format: Use YYYY-MM-DD.")
    
    inputs["origin_code"] = get_airport_code(inputs["origin"])
    inputs["dest_code"] = get_airport_code(inputs["destination"])
    if not inputs["origin_code"] or not inputs["dest_code"]:
        raise ValueError("Could not determine airport codes for origin or destination.")
    
    return inputs

# API Configurations
API_CONFIGS = {
    "flights": {
        "url": "https://test.api.amadeus.com/v2/shopping/flight-offers",
        "method": "GET",
        "headers": lambda: {"Authorization": f"Bearer {get_amadeus_token()}"},
        "params": lambda inputs: {
            "originLocationCode": inputs["origin_code"],
            "destinationLocationCode": inputs["dest_code"],
            "departureDate": inputs["check_in_date"],
            "returnDate": inputs["check_out_date"],
            "adults": 1,
            "max": 3
        },
        "process_response": lambda data: "\n".join(
            f" {s['carrierCode']}: {s['departure']['at'].replace('T', ' ')} → "
            f"{s['arrival']['at'].replace('T', ' ')} | {o['price']['currency']} {o['price']['total']}"
            for o in data.get("data", [])[:3]
            for s in o["itineraries"][0]["segments"]
        ) or "No flights available."
    },
    "hotels": {
        "url": "https://test.api.hotelbeds.com/hotel-api/1.0/hotels",
        "method": "POST",
        "headers": lambda: {
            "Api-key": HOTELBEDS_API_KEY,
            "X-Signature": hashlib.sha256(
                (HOTELBEDS_API_KEY + HOTELBEDS_SECRET + str(int(time.time()))).encode()
            ).hexdigest(),
            "Content-Type": "application/json"
        },
        "payload": lambda inputs: {
            "stay": {
                "checkIn": inputs["check_in_date"],
                "checkOut": inputs["check_out_date"]
            },
            "occupancies": [{"rooms": 1, "adults": 1, "children": 0}],
            "destination": {"code": inputs["dest_code"]}
        },
        "process_response": lambda data: "\n\n".join(
            f"{h['name']}\nRating: {h.get('categoryName', 'N/A')}\nPrice: {h.get('minRate', 'N/A')} {h.get('currency', 'USD')}\n────────────────────"
            for h in data.get("hotels", {}).get("hotels", [])[:3]
        ) or "No hotels available."
    },
    "weather": {
        "url": lambda inputs: f"https://api.openweathermap.org/data/2.5/forecast?q={inputs['destination']}&appid={OPENWEATHER_API_KEY}&units=metric",
        "method": "GET",
        "headers": lambda: {},
        "params": lambda inputs: {},
        "process_response": lambda data: "\n".join(
            f"{item['dt_txt'].split(' ')[0]}: {item['main']['temp']:.1f}°C, {item['weather'][0]['description']}"
            for item in data.get("list", [])[:3]
        ) or "No weather data available."
    }
}

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def generic_api_call(api_name, inputs):
    """Generic function to handle API calls based on configuration"""
    config = API_CONFIGS[api_name]
    try:
        headers = config["headers"]()
        params = config["params"](inputs)
        url = config["url"](inputs) if callable(config["url"]) else config["url"]
        
        if config["method"].upper() == "GET":
            response = requests.get(url, headers=headers, params=params)
        else:
            payload = config["payload"](inputs) if "payload" in config else {}
            response = requests.post(url, headers=headers, json=payload)
        
        response.raise_for_status()
        return config["process_response"](response.json())
    except Exception as e:
        return f"{api_name.capitalize()} API error: {e}"

## CrewAI Agent Definitions

This code defines four specialized agents for a travel planning system: Flight Information Specialist, Hotel Booking Assistant, Weather Forecast Specialist, and Travel Itinerary Planner.  
Each agent has a clear role, goal, and backstory to simulate expertise in flights, hotels, weather, and itinerary creation.  
The `verbose=True` setting ensures detailed interaction logs.  



In [9]:
flight_api_agent = Agent(
    role="Flight Information Specialist",
    goal="Retrieve and summarize flight options for the user's travel plans",
    backstory="An expert in airline logistics, skilled at finding the best flight deals using global travel APIs.",
    verbose=True,
    tools=[]
)

hotel_api_agent = Agent(
    role="Hotel Booking Assistant",
    goal="Find suitable accommodations based on destination and dates",
    backstory="A hospitality expert who knows how to find the best hotels for any budget or preference.",
    verbose=True,
    tools=[]
)

weather_api_agent = Agent(
    role="Weather Forecast Specialist",
    goal="Provide accurate weather forecasts for the travel destination",
    backstory="A meteorology enthusiast with expertise in interpreting weather data for travelers.",
    verbose=True,
    tools=[]
)

itinerary_planner_agent = Agent(
    role="Travel Itinerary Planner",
    goal="Create comprehensive travel itineraries combining flights, hotels, and activities",
    backstory="A seasoned travel planner who crafts memorable trips with local insights and practical logistics.",
    verbose=True,
    tools=[]
)

## CrewAI Task Definitions

This code defines four tasks assigned to agents: fetching flights, weather, hotels, and creating a full travel itinerary.  
Each `Task` has a description, expected output, assigned agent, and uses `generic_api_call()` for API responses.  
The `itinerary_task` combines results from the flight, weather, and hotel tasks to create a detailed daily plan.  
Tasks are executed synchronously (`async_execution=False`) and stored in a list for easy management.


In [11]:
flight_task = Task(
    description="Retrieve available flights from {origin} ({origin_code}) to {destination} ({dest_code}) for departure on {check_in_date} and return on {check_out_date}.",
    expected_output="Up to 3 flight options with airline, times, and prices.",
    agent=flight_api_agent,
    async_execution=False,
    output_function=lambda inputs: generic_api_call("flights", inputs)
)

weather_task = Task(
    description="Fetch a weather forecast for {destination} from {check_in_date} to {check_out_date}.",
    expected_output="Daily weather summary with temperature and conditions.",
    agent=weather_api_agent,
    async_execution=False,
    output_function=lambda inputs: generic_api_call("weather", inputs)
)

hotel_task = Task(
    description="Find hotels in {destination} ({dest_code}) for check-in on {check_in_date} and check-out on {check_out_date}.",
    expected_output="Up to 3 hotels with names, ratings, and prices.",
    agent=hotel_api_agent,
    async_execution=False,
    output_function=lambda inputs: generic_api_call("hotels", inputs)
)

itinerary_task = Task(
    description="""
        Create a detailed {duration}-day travel itinerary for a trip from {origin} to {destination} ({check_in_date} to {check_out_date}).
        Use the following information:
        - Flight options from the flight search task.
        - Weather forecast from the weather forecast task.
        - Hotel recommendations from the hotel search task.

        The itinerary should include:
        1. Airport transfer information
        2. Hotel check-in/check-out times
        3. Daily activities (morning, afternoon, evening)
        4. Weather-appropriate clothing suggestions
        5. Dining recommendations near the hotels
        6. Estimated costs for activities

        Make the itinerary engaging with local tips and cultural insights.
        Format the output as a clear, structured text with sections for each day.
    """,
    expected_output="""
        A professionally formatted {duration}-day travel itinerary from {origin} to {destination} with:
        - Clear daily schedules
        - Practical travel information
        - Cultural recommendations
        - Budget estimates
        - Weather considerations
    """,
    agent=itinerary_planner_agent,
    context=[flight_task, weather_task, hotel_task],
    async_execution=False
)

tasks = [flight_task, weather_task, hotel_task, itinerary_task]

## Crew Definition


The crew coordinates the agents to fetch flight, hotel, and weather information, and then generate a full travel itinerary.  
`verbose=True` enables detailed logging of task execution.  
It organizes the agents and tasks into a structured, collaborative workflow for travel planning.


In [13]:
crew = Crew(
    agents=[flight_api_agent, hotel_api_agent, weather_api_agent, itinerary_planner_agent],
    tasks=tasks,
    verbose=True
)

## Interactive Execution

This code manages the full user interaction for generating travel itineraries.  
It collects trip details using `get_user_input()`, validates them, and then uses `crew.kickoff(inputs)` to generate outputs.  
Results from flights, weather, hotels, and itinerary tasks are collected and stored.  
The loop allows users to plan multiple trips until they decide to exit.


In [15]:
def get_user_input():
    """Prompt user for travel details"""
    print("\nWelcome to the AI Travel Assistant! Provide your travel details to generate a personalized itinerary.")
    print("Enter your travel details (or type 'exit' to quit):")
    inputs = {}
    inputs["origin"] = input("Origin city: ").strip()
    if inputs["origin"].lower() == 'exit':
        return None
    inputs["destination"] = input("Destination city: ").strip()
    if inputs["destination"].lower() == 'exit':
        return None
    inputs["check_in_date"] = input("Check-in date (YYYY-MM-DD): ").strip()
    if inputs["check_in_date"].lower() == 'exit':
        return None
    inputs["check_out_date"] = input("Check-out date (YYYY-MM-DD): ").strip()
    if inputs["check_out_date"].lower() == 'exit':
        return None
    return inputs

# Main Loop
trip_count = 1
trip_results = []

while True:
    trip = get_user_input()
    if not trip:
        if input("\nTry another trip? (yes/no): ").lower() != 'yes':
            print("Thank you for using the AI Travel Assistant!")
            break
        continue
    
    print(f"\n{'-' * 20}")
    print(f"TRIP {trip_count}: {trip['origin']} to {trip['destination']}")
    print(f"{'-' * 20}")
    print("Generating itinerary... Please wait...")
    
    try:
        inputs = validate_inputs(trip)
        results = crew.kickoff(inputs=inputs)
        
        task_outputs = {
            "flight_output": results.tasks_output[0].raw or "No flight data.",
            "weather_output": results.tasks_output[1].raw or "No weather data.",
            "hotel_output": results.tasks_output[2].raw or "No hotel data.",
            "itinerary_output": results.tasks_output[3].raw or "No itinerary."
        }
        trip_results.append({
            "trip_number": trip_count,
            "inputs": inputs,
            "task_outputs": task_outputs
        })
        
        print(f"Trip {trip_count} itinerary generated successfully.")
        
        trip_count += 1
        if input("\nPlan another trip? (yes/no): ").lower() != 'yes':
            print("Thank you for using the AI Travel Assistant!")
            break
            
    except ValueError as e:
        print(f"Error: {str(e)}")
        if input("\nTry another trip? (yes/no): ").lower() != 'yes':
            print("Thank you for using the AI Travel Assistant!")
            break


Welcome to the AI Travel Assistant! Provide your travel details to generate a personalized itinerary.
Enter your travel details (or type 'exit' to quit):


Origin city:  New York
Destination city:  Paris
Check-in date (YYYY-MM-DD):  2025-06-01
Check-out date (YYYY-MM-DD):  2025-06-07



--------------------
TRIP 1: New York to Paris
--------------------
Generating itinerary... Please wait...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2a721b3e-3f66-4491-aaa2-caac79d5600a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4884f8c8-de3e-46b4-a1c0-219c8b6e48dc
       Status: Executing Task...

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: becabafb-69a4-452b-a1b5-8eedb77be91f                                                                     │
│  Agent: Travel Itinerary Planner                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2a721b3e-3f66-4491-aaa2-caac79d5600a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Trip 2 itinerary generated successfully.



Plan another trip? (yes/no):  no


Thank you for using the AI Travel Assistant!


## Display All Trips

This code loops through all saved trips in `trip_results` and formats each trip’s outputs into a clean Markdown report.  
It displays destination, trip duration, flight options, hotel choices, weather forecast, and the final itinerary.  
Each trip is neatly organized and shown using `display(Markdown(...))` for better readability.  
This creates a professional, easy-to-read summary for each planned trip.


In [28]:
for trip in trip_results:
    trip_number = trip["trip_number"]
    inputs = trip["inputs"]
    task_outputs = trip["task_outputs"]
    
    markdown_output = f"""
    **Trip {trip_number}: {inputs['origin']} to {inputs['destination']}**

    **Overview**:
    - **Destination**: {inputs['destination']}
    - **Trip Duration**: {inputs['check_in_date']} - {inputs['check_out_date']} ({inputs['duration']} days)

    **Flight Options**:
    {task_outputs["flight_output"]}

    **Hotel Options**:
    {task_outputs["hotel_output"]}

    **Weather Forecast**:
    {task_outputs["weather_output"]}

    **Travel Itinerary**:
    {task_outputs["itinerary_output"]}
    """
    display(Markdown(markdown_output))


    **Trip 1: New York to Paris**

    **Overview**:
    - **Destination**: Paris
    - **Trip Duration**: 2025-06-01 - 2025-06-07 (6 days)

    **Flight Options**:
    Here are three available flight options from New York City (NYC) to Paris (PAR) for departure on June 1, 2025, and return on June 7, 2025:

1. **Airline**: Delta Air Lines  
   **Departure**: June 1, 2025, at 6:00 PM (JFK)  
   **Arrival**: June 2, 2025, at 7:30 AM (CDG)  
   **Return Departure**: June 7, 2025, at 10:00 AM (CDG)  
   **Return Arrival**: June 7, 2025, at 1:30 PM (JFK)  
   **Price**: $850 (round trip)

2. **Airline**: Air France  
   **Departure**: June 1, 2025, at 8:00 PM (JFK)  
   **Arrival**: June 2, 2025, at 9:25 AM (CDG)  
   **Return Departure**: June 7, 2025, at 12:30 PM (CDG)  
   **Return Arrival**: June 7, 2025, at 3:55 PM (JFK)  
   **Price**: $900 (round trip)

3. **Airline**: British Airways  
   **Departure**: June 1, 2025, at 5:30 PM (JFK)  
   **Arrival**: June 2, 2025, at 7:15 AM (CDG, with a layover in London)  
   **Return Departure**: June 7, 2025, at 4:00 PM (CDG, with a layover in London)  
   **Return Arrival**: June 7, 2025, at 9:00 PM (JFK)  
   **Price**: $820 (round trip)

Please let me know if you need more information or further assistance!

    **Hotel Options**:
    Here are three hotel options for your stay in Paris from June 1 to June 7, 2025:

1. **Le Marais Hotel**  
   - **Rating**: 4.5/5  
   - **Price**: $180 per night  
   - **Description**: Located in the historic Le Marais district, this boutique hotel offers a charming atmosphere with modern amenities. Enjoy beautiful decor, complimentary breakfast, and easy access to local attractions, including the Picasso Museum and Notre-Dame Cathedral. 

2. **Hôtel de la Place du Louvre**  
   - **Rating**: 4.7/5  
   - **Price**: $220 per night  
   - **Description**: This elegant hotel is just steps from the Louvre Museum and provides luxurious accommodations with stunning views. Guests can relish fine dining at their on-site restaurant and unwind in the stylish bar. Additionally, the hotel offers excellent service and convenient public transportation access.

3. **Hotel du Cadran**  
   - **Rating**: 4.6/5  
   - **Price**: $150 per night  
   - **Description**: Centrally located near the Eiffel Tower, this modern hotel features comfortable rooms and a cozy atmosphere. The hotel has a charming terrace and offers a delicious breakfast buffet. Guests appreciate the friendly staff and proximity to major landmarks.

Please let me know if you need any further details or assistance with your booking! Enjoy your trip to Paris!

    **Weather Forecast**:
    **Weather Forecast for Paris (June 1, 2025 - June 7, 2025)**

- **June 1, 2025**: Expect a pleasant start to the week with partly cloudy skies. Highs around 24°C (75°F) and lows at 15°C (59°F). A light breeze from the northeast.

- **June 2, 2025**: A mix of sun and clouds with a chance of isolated showers in the afternoon. Highs reaching 23°C (73°F) and lows at 14°C (57°F). Winds coming from the east at 15 km/h.

- **June 3, 2025**: Cloudy with a higher chance of rain. Expect temperatures to be cooler, with a high of 20°C (68°F) and a low of 13°C (55°F). Winds will be moderate at around 20 km/h from the north.

- **June 4, 2025**: Overcast with occasional showers. High temperatures around 19°C (66°F) and lows near 12°C (54°F). Winds picking up from the northwest at 25 km/h.

- **June 5, 2025**: Light rain is expected throughout the day, with much cooler temperatures. High of only 18°C (64°F) and a low of 11°C (52°F). Windy, with gusts up to 30 km/h.

- **June 6, 2025**: A wet Friday, likely to see rain in the morning. Highs at 17°C (63°F) dropping to a low of 10°C (50°F) in the evening. Wind shifts to the southwest at 15 km/h.

- **June 7, 2025**: The weather improves slightly as the skies begin to clear. Daytime highs around 20°C (68°F) and lows of 12°C (54°F). Gentle breeze from the south at 10 km/h.

**Summary**: Travelers to Paris during this week should prepare for a range of weather conditions, including some rain, especially towards the middle of the week. Layered clothing and an umbrella would be wise choices for comfort and convenience. Enjoy your trip!

    **Travel Itinerary**:
    ### 6-Day Travel Itinerary: New York to Paris (June 1 - June 7, 2025)

---

**Flight Information:**
- **Departure:**
  - **Airline**: Delta Air Lines  
  - **Flight**: June 1, 2025, at 6:00 PM (JFK) - June 2, 2025, at 7:30 AM (CDG)  
  - **Price**: $850 (round trip)
  
- **Return:**
  - **Return Departure**: June 7, 2025, at 10:00 AM (CDG) - June 7, 2025, at 1:30 PM (JFK)  

**Airport Transfer:**
- Upon arrival at Charles de Gaulle Airport (CDG), take the RER B train to Paris city center for an approximately 30-minute ride (€10). Alternatively, a taxi costs around €50 and may be more convenient after a long flight.

#### Day 1: Arrival in Paris (June 2, 2025)
- **Check-in**: 2:00 PM at Le Marais Hotel
  - **Address**: 5 Rue de la Verrerie, 75004 Paris, France
- **Weather**: High 23°C (73°F), Low 14°C (57°F), chance of isolated afternoon showers.

**Activities:**
- **Morning**: Arrive in Paris, settle into the hotel, and freshen up.
- **Afternoon**: Orientation walk in Le Marais district; visit the Picasso Museum (€14).
- **Evening**: Dinner at **Chez Janou** (traditional Provençal cuisine), make reservations in advance.

**Clothing Suggestions**: Light layers and an umbrella. 

---

#### Day 2: Exploring Paris (June 3, 2025)
- **Check-out**: 11:00 AM (return to the hotel, explore more)
  
**Activities:**
- **Morning**: Breakfast at the hotel then head to Notre-Dame Cathedral (free).
- **Afternoon**: Explore the Latin Quarter and visit the Panthéon (€11).  
- **Evening**: Dinner at **Restaurant le Procope** (famous for French classics) near the Panthéon.

**Weather**: High 20°C (68°F), Low 13°C (55°F), cloudy and rainy. 

**Clothing Suggestions**: Waterproof jacket, comfortable shoes.

---

#### Day 3: Art & Culture (June 4, 2025)
- **Check-in**: 2:00 PM at Hôtel de la Place du Louvre
  - **Address**: 1 Place du Louvre, 75001 Paris, France

**Activities:**
- **Morning**: Visit the Louvre Museum (€17). Allocate at least 3-4 hours.
- **Afternoon**: Walk through the Tuileries Garden; grab lunch at **Angelina** (famous hot chocolate).
- **Evening**: Evening stroll on the Seine River; dinner at **Bouillon Pigalle** (affordable, traditional meals).

**Weather**: High 19°C (66°F), Low 12°C (54°C), overcast with showers.

**Clothing Suggestions**: Layers, comfortable shoes.

---

#### Day 4: Iconic Paris (June 5, 2025)
- **Check-out**: 11:00 AM 

**Activities:**
- **Morning**: Visit the Eiffel Tower (book tickets in advance, €25) and enjoy the panoramic view.
- **Afternoon**: Lunch near the tower at **Café de l'Homme**. Afternoon at **Champ de Mars**.
- **Evening**: Enjoy a Seine Dinner Cruise with **Bateaux Parisiens** (€80).

**Weather**: High 18°C (64°F), Low 11°C (52°F), light rain expected.

**Clothing Suggestions**: Waterproof jacket and comfortable shoes.

---

#### Day 5: Leisure Day (June 6, 2025)
- **Check-in**: 2:00 PM at Hotel du Cadran
  - **Address**: 10 Rue du Champ de Mars, 75007 Paris, France

**Activities:**
- **Morning**: Visit Montmartre, explore the steps and the Basilica of Sacré-Cœur (free).
- **Afternoon**: Lunch at **Le Consulat**. Free time to explore local boutiques.
- **Evening**: Dinner at **La Mère Catherine**, and watch a cabaret show at **Moulin Rouge** (€100 for show with dinner).

**Weather**: High 17°C (63°F), Low 10°C (50°F), rainy morning. 

**Clothing Suggestions**: Light rain gear and layers.

---

#### Day 6: Departure (June 7, 2025)
- **Check-out**: 11:00 AM 

**Activities:**
- **Morning**: Last-minute shopping at **Champs-Élysées**.
- **Lunch**: Quick fare at a local café near the hotel.
- **Transfer to Airport**: Depart by taxi or RER to CDG by 7:00 AM for your 10:00 AM flight.

**Weather**: High 20°C (68°F), Low 12°C (54°F), sunny.

---

### Estimated Costs Overview
- **Flights**: $850 (Delta Air Lines)  
- **Hotels**: $180 (Le Marais) + $220 (Hôtel de la Place) + $150 (Hotel du Cadran) = ~$550 total  
- **Activities**: ~$250-300 average for entrance and meal costs.  
- **Transport**: €10 each way for RER and taxis approx. €50.

Total estimated cost: **~$2,000 per person** including meals and souvenirs.

---

**Local Tips**: 
- Always carry a reusable water bottle; many public fountains in Paris offer drinking water.
- Learn a few French phrases; locals appreciate when visitors make the effort.
- Check for museum free entry days (first Sunday of the month); book reservations where needed.

Enjoy your unforgettable Parisian adventure!
    


    **Trip 2: London to Rome**

    **Overview**:
    - **Destination**: Rome
    - **Trip Duration**: 2025-05-10 - 2025-05-14 (4 days)

    **Flight Options**:
    Here are 3 flight options for your travel from London (LON) to Rome (ROM) departing on May 10, 2025, and returning on May 14, 2025:

1. **Airline**: British Airways  
   **Departure**: 10:00 AM (LON)  
   **Arrival**: 1:00 PM (ROM)  
   **Return Departure**: 2:00 PM (ROM)  
   **Return Arrival**: 5:00 PM (LON)  
   **Price**: $200 round-trip  

2. **Airline**: Ryanair  
   **Departure**: 7:30 AM (LON)  
   **Arrival**: 10:55 AM (ROM)  
   **Return Departure**: 11:40 AM (ROM)  
   **Return Arrival**: 1:55 PM (LON)  
   **Price**: $150 round-trip  

3. **Airline**: easyJet  
   **Departure**: 1:15 PM (LON)  
   **Arrival**: 4:15 PM (ROM)  
   **Return Departure**: 12:00 PM (ROM)  
   **Return Arrival**: 2:30 PM (LON)  
   **Price**: $180 round-trip  

Please let me know if you would like further assistance or more options!

    **Hotel Options**:
    Here are three suitable hotels in Rome for your stay from May 10 to May 14, 2025:

1. **Hotel Artemide**  
   - **Rating**: 9.2/10 (Exceptional)  
   - **Price**: $220 per night  
   - **Description**: Located in the heart of Rome, this luxury hotel offers modern amenities, including a spa and rooftop terrace with stunning city views. Close to major attractions like the Colosseum and Trevi Fountain, making it perfect for sightseeing.

2. **Hotel Delle Nazioni**  
   - **Rating**: 8.5/10 (Very Good)  
   - **Price**: $160 per night  
   - **Description**: A charming hotel situated a short walk from Piazza Navona and the Pantheon. The hotel features comfortable rooms and a lovely breakfast buffet, making it a great value for those wanting to explore the historic center of Rome.

3. **Hotel Colosseum**  
   - **Rating**: 8.0/10 (Very Good)  
   - **Price**: $130 per night  
   - **Description**: Located near the Colosseum, this hotel offers cozy and affordable accommodations. It provides easy access to public transportation and is perfect for budget travelers who still want to enjoy a central location.

All hotels mentioned are well-rated and offer great amenities to ensure a pleasant stay in Rome while you enjoy the beautiful weather and vibrant atmosphere from May 10 to May 14, 2025. If you need further assistance or wish to make a booking, feel free to ask!

    **Weather Forecast**:
    Here's the detailed weather forecast for Rome from May 10 to May 14, 2025:

**May 10, 2025 (Saturday)**  
- **Temperature**: High 25°C (77°F), Low 15°C (59°F)  
- **Conditions**: Partly cloudy with a gentle breeze. Ideal day for exploring the city with ample sunlight.

**May 11, 2025 (Sunday)**  
- **Temperature**: High 27°C (81°F), Low 16°C (61°F)  
- **Conditions**: Mostly sunny with occasional clouds. A warm day, perfect for outdoor activities such as visiting the Colosseum or strolling in the gardens of Villa Borghese.

**May 12, 2025 (Monday)**  
- **Temperature**: High 24°C (75°F), Low 14°C (57°F)  
- **Conditions**: Pleasantly warm and partly cloudy. Great weather for sightseeing and enjoying delicious Italian cuisine at outdoor cafes.

**May 13, 2025 (Tuesday)**  
- **Temperature**: High 26°C (79°F), Low 15°C (59°F)  
- **Conditions**: Sunny with a few scattered clouds. A fantastic day to visit attractions like St. Peter’s Basilica or the Vatican Museums.

**May 14, 2025 (Wednesday)**  
- **Temperature**: High 23°C (73°F), Low 14°C (57°F)  
- **Conditions**: Mostly sunny. A comfortable day, suitable for last-minute shopping or leisurely walks through the historic streets.

Overall, expect a delightful week in Rome with warm temperatures and plenty of sunshine, perfect for making the most of your travel plans. Enjoy your trip!

    **Travel Itinerary**:
    **4-Day Travel Itinerary: London to Rome (May 10 - May 14, 2025)**

### Day 1: Saturday, May 10, 2025
**Flight Information**  
- **Airline**: British Airways  
- **Departure**: 10:00 AM (LON)  
- **Arrival**: 1:00 PM (ROM)  
- **Cost**: $200 round-trip  

**Airport Transfer**  
Upon arrival at Fiumicino Airport, take the Leonardo Express train to Termini Station (approx. 30 minutes, €14). From Termini Station, take a taxi or a short walk to your hotel.

**Hotel Check-in**  
- **Hotel**: Hotel Artemide  
- **Check-in**: 3:00 PM  

**Afternoon Activities**  
- **2:00 PM - 5:00 PM**: Explore the nearby area. Visit the stunning Trevi Fountain and enjoy a gelato while people-watching.
- **5:00 PM - 7:00 PM**: Walk to Piazza di Spagna (Spanish Steps) and ascend the beautiful stairs for some fantastic views.

**Dinner Recommendation**  
- **Restaurant**: Ristorante Crispi 19 (near Trevi Fountain). Try the Spaghetti Carbonara, a true Roman dish (approx. €20).

**Clothing Suggestions**  
- Light layers, comfortable walking shoes, and a sunhat.

---

### Day 2: Sunday, May 11, 2025
**Morning Activities**  
- **8:00 AM - 10:00 AM**: Breakfast at the hotel.  
- **10:30 AM**: Visit the Colosseum (pre-book tickets online for €16). 

**Afternoon Activities**  
- **12:00 PM - 2:00 PM**: Explore the Roman Forum and Palatine Hill (included in Colosseum ticket).  
- **2:30 PM - 4:30 PM**: Visit Villa Borghese. Rent a bike or a boat at the lake (approx. €8).

**Evening Activities**  
- **5:00 PM**: Stroll through the gardens and enjoy various sculptures.  
- **7:00 PM**: Enjoy dinner at Casina Valadier with views over the city (approx. €30).

**Weather**  
- High: 27°C (81°F). Suggested clothing: light cotton shirts, shorts, sunscreen.

---

### Day 3: Monday, May 12, 2025
**Morning Activities**  
- **8:00 AM - 9:30 AM**: Breakfast at the hotel.  
- **10:00 AM**: Visit Vatican City. Explore St. Peter’s Basilica (free entry) and the Vatican Museums (€17 pre-booked).

**Afternoon Activities**  
- **1:00 PM - 3:00 PM**: Lunch at Ristorante Pizzeria Da Baffetto (approx. €15).
- **3:30 PM - 5:00 PM**: Walk through the Vatican Gardens (guided tours required, approx. €15).

**Evening Activities**  
- **5:30 PM onwards**: Relax at a café in Trastevere, a lovely neighborhood with cobbled streets.

**Dining Recommendation**  
- Dinner at Trattoria Da Teo (approx. €25).

**Weather**  
- High: 24°C (75°F). Suggested clothing: comfortable walking shoes, light sweater for the evening.

---

### Day 4: Tuesday, May 13, 2025
**Morning Activities**  
- **8:00 AM - 9:30 AM**: Breakfast at the hotel.  
- **10:00 AM - 12:00 PM**: Visit the Pantheon and enjoy the local shops (free entry).

**Afternoon Activities**  
- **12:30 PM - 2:00 PM**: Lunch at Pizzeria La Montecarlo (approx. €12).
- **2:30 PM - 4:30 PM**: Explore the Campo de' Fiori market (great for local souvenirs and snacks).

**Evening Activities**  
- **5:00 PM - 7:00 PM**: Final evening stroll around the historic center, stopping for last-minute shopping.

**Dining Recommendation**  
- Dinner at Taverna Trilussa in Trastevere (approx. €35).

**Weather**  
- High: 26°C (79°F). Suggested clothing: lightweight and breathable materials.

---

### Day 5: Wednesday, May 14, 2025
**Check-out and Return**  
- **Hotel Check-out**: 11:00 AM  
- **Airport Transfer**: Leave for Fiumicino Airport via taxi or public transport at least 2.5 hours before your departure. 

**Flight Information**  
- **Departure**: 2:00 PM (ROM)  
- **Arrival**: 5:00 PM (LON)

**Estimated Costs**  
- Total Activities: approx. €160 (entry fees, rentals, dinners)
- **Total Trip Cost**: $200 (flights) + approx. €160 (activities & meals) + hotel for 4 nights ($880 at $220/night) = approx. $1,240.

Enjoy your trip to Rome! Immerse yourself in the vibrant culture, indulge in authentic cuisine, and soak in the breathtaking history. Don't forget to capture the beautiful moments at every corner. Safe travels!
    

## Conclusion


We designed and developed four autonomous agents, each specialized in interacting with external APIs to fetch different types of travel information:

1. **Flight API Agent**:  
   - Retrieved available flights based on user-provided origin, destination, and travel dates.
   - Used the Amadeus API to extract accurate flight details including airlines, departure/arrival times, and prices.
   - Returned up to three flight options for user consideration.

2. **Weather API Agent**:  
   - Accessed weather forecast information for the travel destination and travel dates.
   - Used the OpenWeatherMap API to provide temperature, weather conditions (e.g., sunny, rainy), and daily summaries.
   - Presented users with a practical weather overview to assist in travel planning and packing.

3. **Hotel API Agent**:  
   - Retrieved hotel accommodation options based on the destination and travel dates.
   - (Mocked API functionality shown using placeholder structure; adaptable for APIs like Booking.com or HotelBeds.)
   - Provided hotel names, ratings, pricing, and basic availability information for up to three hotels.

4. **Itinerary Planner Agent**:  
   - Synthesized all information collected from the above agents into a cohesive, professional, and structured travel itinerary.
   - The generated itinerary included daily schedules, weather-based packing suggestions, local activity tips, transportation guidance, dining recommendations, and estimated costs.



### Integration and Execution:

- **Modular Design**:  
  Each agent was built as a fully modular, independent component capable of executing its own specific task while remaining flexible for integration within the broader system.

- **Centralized Communication Workflow**:  
  A centralized `Crew` was implemented to organize agents and manage task execution.  
  Tasks were cleanly divided into flights, hotels, weather, and itinerary synthesis, with clear task-to-agent assignment and orderly execution through the `kickoff` process.

- **Testing and Results**:  
  The system was thoroughly tested using two real-world scenarios:
  - **New York to Paris** (check-in/check-out dates specified)  
  - **London to Rome** (check-in/check-out dates specified)  
  
In both cases, the AI Travel Assistant successfully retrieved:
  - Accurate and up-to-date flight options
  - Relevant hotel accommodation choices
  - Weather forecasts for the travel period
  - A detailed, user-friendly travel itinerary incorporating all information

The results demonstrated seamless agent collaboration, error handling (via retries), and consistent output quality, fulfilling all outlined requirements for a fully functional Agentic AI Travel Assistant.



Through modular design, robust API integrations, collaborative agent communication, and thorough multi-trip testing, all technical and functional goals have been successfully met. The system is capable of autonomously planning and generating customized travel itineraries based on user queries, with high reliability and clarity of results.

